## The common experiment functions
### The functions are used to perform model optimization & hyper parameters tunning
### After the tunning, the best parameters a combined to come up with the best model

In [6]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [7]:
#---------------------------------------------------------------------
# Function to create a default configuration for our neural network model
# The configuration can be overridden while creating the model
#
def base_model_config():
    model_config = {
        "HIDDEN_NODES" : [32,64],
        "HIDDEN_ACTIVATION" : "relu",
        "OUTPUT_NODES" : 3,
        "OUTPUT_ACTIVATION" : "sigmoid",
        "WEIGHTS_INITIALIZER" : "random_normal",
        "BIAS_INITIALIZER" : "zeros",
        "NORMALIZATION" : "none",
        "OPTIMIZER" : "adam",
        "LEARNING_RATE" : 0.001,
        "REGULARIZER" : None,
        "DROPOUT_RATE" : 0.0,
        "EPOCHS" : 10,
        "BATCH_SIZE" : 16,
        "VALIDATION_SPLIT" : 0.2,
        "VERBOSE" : 0,
        "LOSS_FUNCTION" : "binary_crossentropy",
        "METRICS" : ["accuracy"]
    }
    return model_config

In [8]:
#---------------------------------------------------------------------
# Function to create an optimizer based on the optimizer name and learning rate
#---------------------------------------------------------------------
def get_optimizer(optimizer_name, learning_rate):
    # adagrad, adam, rmsprop, and sgd
    optimizer = None

    if optimizer_name == "adagrad":
        optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate)
    elif optimizer_name == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "rmsprop":
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

    return optimizer